# 10wk-2: (추천시스템) – optimizer 사용 고급, 모델링 전략, MF-based

추천시스템

최규빈  
2025-05-12

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/09wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>